In [1]:
# 通过openai包直接调用其他的大模型，例如此处的千问大模型
import os
from openai import OpenAI
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key=os.getenv("DASHSCOPE_API_KEY"), 
    # api_key='sk-25186dbe2f984cbcba48ffca2c81a78f',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)
completion = client.chat.completions.create(
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '你是谁？'}],
    )
    
print(completion.model_dump_json())

{"id":"chatcmpl-536b5211-5e83-9680-a89a-caf254b0d3db","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"我是通义千问，由阿里云开发的AI助手。我可以回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1729739359,"model":"qwen-plus","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":33,"prompt_tokens":22,"total_tokens":55,"completion_tokens_details":null,"prompt_tokens_details":null}}


In [18]:
# 通过langchain的langchain_openai包直接调用其他的大模型，例如此处的千问大模型
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key=os.getenv("ZHIPU_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
#交流缓存记忆，将沟通的信息进行存储
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke'),
  AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄")],
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"}

In [34]:
#MessagesPlaceholder的应用
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder("history"),
    ("human", "{question}")
])

prompt.invoke({
    "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
    "question": "now multiply that by 4"
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.'), HumanMessage(content="what's 5 + 2"), AIMessage(content='5 + 2 is 7'), HumanMessage(content='now multiply that by 4')])

In [37]:
#存储记忆的提取
memory.save_context({"input": "你好！"} ,{"outputs": "你好，有什么我可以帮助你的吗？"})
chat_history = memory.load_memory_variables({"input": "..."})
print(chat_history["chat_history"])

[HumanMessage(content='tell me a joke'), AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？'), HumanMessage(content='你好！'), AIMessage(content='你好，有什么我可以帮助你的吗？')]


In [39]:
# 调用智谱大模型进行对话
from langchain_community.chat_models import ChatZhipuAI
zhipuai_chat = ChatZhipuAI(
        temperature=0.5,
        api_key=os.getenv("ZHIPU_API_KEY"),
        model="glm-4",
    )
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="Your role is a poet."),
    HumanMessage(content="三打白骨精题材."),
]
# 打印结果
response = zhipuai_chat.invoke(messages)
print(response.content)

《三打白骨精》是中国古典名著《西游记》中的一个著名章节，讲述了唐僧师徒四人在取经路上遇到善于变化的白骨精的故事。以下是一首以这个题材为基础的诗：

西行路险多妖氛，
白骨精藏巧机关。
三番变化迷人眼，
一试禅心见佛缘。

初化村姑施怜悯，
再成老妇演悲欢。
三作将军威风凛，
真假难辨雾中烟。

悟空火眼识妖怪，
金箍棒下不容宽。
护师取经历磨难，
骨碎精消证菩提。


In [51]:
# 通过agent 查询网站资料回答问题
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=5)]
prompt = hub.pull("hwchase17/react")
llm.max_tokens = 5000 
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True,verbose=True)
agent_executor.invoke({"input": "撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字"})

c:\Users\admin\anaconda3\envs\qianwen\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
这个问题的答案需要较长的文字输出，并且要围绕“全面从严治党，永远在路上”的主题。我需要通过搜索来收集相关信息和案例，以便撰写这篇报告。

Action: tavily_search_results_json
Action Input: “全面从严治党”案例及成效

Observation[{'url': 'http://dangjian.people.com.cn/GB/n1/2021/1111/c117092-32279396.html', 'content': '全面从严治党，是我们党进行具有许多新的历史特点的伟大斗争的根本保证。. 管党治党是事关民心向背、执政根基的政治责任。. 习近平总书记强调，"从严治党，必须增强管党治党意识、落实管党治党责任"。. 各级党委作为管党治党的领导主体、落实主体'}, {'url': 'https://www.htxy.edu.cn/kjc/info/1069/2384.htm', 'content': '办好中国的事情，关键在党，关键在党要管党，从严治党。习近平总书记在庆祝中国共产党成立一百周年大会上深刻指出："我们党历经千锤百炼而朝气蓬勃，一个很重要的原因就是我们始终坚持党要管党、全面从严治党"。回望党的百年奋斗历程，党之所以能够始终保持旺盛生命力和坚强战斗力'}]通过搜索结果，我收集到了一些关于“全面从严治党”的案例和成效。接下来，我将基于这些资料，着手撰写报告。

Action: tavily_search_results_json
Action Input: “全面从严治党”的重要性和必要性

Observation[{'url': 'http://dangjian.people.com.cn/n1/2021/1013/c117092-32251702.html', 'content': '深刻认识全面从严治党的重大意义. 全面从严治党，是党的十八大以来，以习近平同志为核心的党中央根据新的历史条件下党的建设面临的新情况和新问题，深刻总结历史上党通过自我革命保持先进性和纯洁性并领导各项事业胜利前进的宝贵经验，对管党治党'}, {'url': 'http://www.qstheory.cn/qshyjx/2021-10/13/c_1127951

{'input': '撰写一篇“全面从严治党，永远在路上”为主题的报告，字数不少于1000字',
 'output': '**\n全面从严治党是我们党的一项长期战略任务，必须始终坚持以习近平新时代中国特色社会主义思想为指导，不断完善党内法规制度，加强党内监督，深化党风廉政建设和反腐败斗争，推动党的建设新的伟大工程不断深化发展。通过本报告的阐述和分析，我们提出了加强理论学习、完善监督机制、强化责任担当、深化廉政建设和推动制度创新等具体措施，以确保党始终保持旺盛的生命力和坚强的战斗力，为实现中华民族伟大复兴的中国梦提供坚强保障。全面从严治党，永远在路上。'}